In [ ]:
import warnings
warnings.filterwarnings('ignore')
import numpy as np
from PIL import Image
import tensorflow as tf
import streamlit as st


@st.cache
def load_image(image_file):
    img = Image.open(image_file)
    return np.array(img)


def nuclei_detection(image):
    """
    This functions takes an image as input and returns segmented image as output
    Input : Input Image
    Output : Input Image, Segmented Image
    """
    # Reading and processing Image
    image = tf.image.convert_image_dtype(image, tf.float32)
    image = tf.image.resize(image, [256, 256])
    # Importing HRNet model
    model = tf.keras.models.load_model("hrnet_model.h5")
    # Predicting Semgentaions on Image using pretrained model
    pred_mask = model.predict(image[np.newaxis, :, :, :])
    # Importing quantized HRNet model and predicting segmentations
    interpreter = tf.lite.Interpreter(model_path="quant_hrnet_model.tflite")
    interpreter.allocate_tensors()
    interpreter.set_tensor(interpreter.get_input_details()[0]['index'], image[np.newaxis,:,:,:])
    interpreter.invoke()
    pred_mask_qh = interpreter.get_tensor(interpreter.get_output_details()[0]['index'])
    return tf.keras.preprocessing.image.array_to_img(image), pred_mask, pred_mask_qh


def main():
    st.title("Nuclei Detection through Image Segmentation")
    html_temp = """
  <div style="background-color:tomato;padding:10px">
  <h2 style="color:white;text-align:center;">Streamlit Nuclei Detection App </h2>
  </div>
  """
    st.markdown(html_temp, unsafe_allow_html=True)
    file = st.file_uploader("Upload PNG Image", type=['png'])

    if file is not None:
        img_ = load_image(file)
        img, pred_mask, pred_mask_qh = nuclei_detection(img_)
        st.image(img, caption='Original Image')
        st.image(pred_mask[0, :, :, 0], caption='HRNet Prediction')
        st.image(pred_mask_qh[:, :, 0], caption='Quantized HRNet Prediction')


if __name__ == '__main__':
    main()